In [1]:
import numpy as np
import pandas as pd
from predictor.test_predictor import RandomForestPredictor
from predictor.train_models import train_linear_regression

In [2]:
model = RandomForestPredictor()
model.train_and_save_model() 

Training model for station: PANC
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Best parameters for station PANC: {'n_estimators': 100}
Training model for station: KBOI
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] END ...................................n_estimators=100; total time=   7.7s


KeyboardInterrupt: 

In [ ]:
from predictor.utils import stations_list
for station in stations_list:
    print(model.predict(f"../data/restructured_simple/{station}.csv",station))

In [ ]:
data = pd.read_csv("regression_data/PANC.csv")

In [ ]:
from analysis.evaluate_model import evaluate_model_years

In [ ]:
evaluate_model_years(2018,2022, predictor=LinearRegressionPredictor())